In [0]:
%sql
SELECT * FROM `hive_metastore`.`default`.`earthquake_data` limit 10;

Search Parameters,Year,Mo,Dy,Hr,Mn,Sec,Tsu,Vol,Location Name,Latitude,Longitude,Focal Depth (km),Mag,MMI Int,Deaths,Death Description,Missing,Missing Description,Injuries,Injuries Description,Damage ($Mil),Damage Description,Houses Destroyed,Houses Destroyed Description,Houses Damaged,Houses Damaged Description,Total Deaths,Total Death Description,Total Missing,Total Missing Description,Total Injuries,Total Injuries Description,Total Damage ($Mil),Total Damage Description,Total Houses Destroyed,Total Houses Destroyed Description,Total Houses Damaged,Total Houses Damaged Description
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,1500.0,1.0,4.0,null,null,null,null,null,CHINA: YUNNAN PROVINCE: YILIANG,24.9,103.1,null,null,9.0,10000.0,4.0,null,null,null,null,null,null,null,null,null,null,10000.0,4.0,null,null,null,null,null,null,null,null,null,null
null,1500.0,null,null,null,null,null,147.0,null,HAWAII,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.0,null,null,null,null,null,null,null,null,null,null
null,1501.0,1.0,19.0,null,null,null,null,null,CHINA: SHAANXI PROVINCE: CHAOYI,34.8,110.1,null,7.0,9.0,400.0,3.0,null,null,null,null,null,4.0,5400.0,4.0,null,null,400.0,3.0,null,null,null,null,null,4.0,5400.0,4.0,null,null
null,1501.0,9.0,24.0,null,null,null,null,null,INDIA: KASHMIR: SRINAGAR,34.1,74.8,null,null,7.0,null,null,null,null,null,null,null,2.0,null,2.0,null,null,null,null,null,null,null,null,null,2.0,null,2.0,null,null
null,1501.0,null,null,null,null,null,null,null,GREECE: CRETE,35.5,25.0,null,null,10.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,1501.0,null,null,null,null,null,null,null,MYANMAR (BURMA): AVA (INNWA),21.8,96.0,null,null,null,null,null,null,null,null,null,null,1.0,null,null,null,null,null,null,null,null,null,null,null,1.0,null,null,null,null
null,1502.0,3.0,26.0,13.0,null,0.0,null,null,BALKANS NW: CROATIA,45.8,16.1,null,null,10.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,1502.0,10.0,17.0,null,null,null,null,null,CHINA: HENAN PROVINCE: PUCHENG,35.7,115.3,null,6.5,8.0,50.0,1.0,null,null,null,null,null,2.0,null,null,null,null,50.0,1.0,null,null,null,null,null,null,null,null,null,null
null,1502.0,null,null,null,null,null,null,null,DOMINICAN REPUBLIC: SANTO DOMINGO,18.3,-69.5,null,null,null,null,null,null,null,null,null,null,3.0,null,null,null,null,null,null,null,null,null,null,null,3.0,null,null,null,null


In [0]:
from pyspark.sql import SparkSession

df = spark.sql("SELECT * FROM hive_metastore.default.earthquake_data")

df_cleaned = df.drop("Search Parameters")

df_cleaned.write.mode("overwrite").saveAsTable("hive_metastore.default.earthquake_data")

df_cleaned.toPandas().head(5)


,Year,Mo,Dy,Hr,Mn,Sec,Tsu,Vol,Location Name,Latitude,Longitude,Focal Depth (km),Mag,MMI Int,Deaths,Death Description,Missing,Missing Description,Injuries,Injuries Description,Damage ($Mil),Damage Description,Houses Destroyed,Houses Destroyed Description,Houses Damaged,Houses Damaged Description,Total Deaths,Total Death Description,Total Missing,Total Missing Description,Total Injuries,Total Injuries Description,Total Damage ($Mil),Total Damage Description,Total Houses Destroyed,Total Houses Destroyed Description,Total Houses Damaged,Total Houses Damaged Description
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1500.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,CHINA: YUNNAN PROVINCE: YILIANG,24.9,103.1,NaN,NaN,9.0,10000.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10000.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1500.0,NaN,NaN,NaN,NaN,NaN,147.0,NaN,HAWAII,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1501.0,1.0,19.0,NaN,NaN,NaN,NaN,NaN,CHINA: SHAANXI PROVINCE: CHAOYI,34.8,110.1,NaN,7.0,9.0,400.0,3.0,NaN,NaN,NaN,NaN,NaN,4.0,5400.0,4.0,NaN,NaN,400.0,3.0,NaN,NaN,NaN,NaN,NaN,4.0,5400.0,4.0,NaN,NaN
4,1501.0,9.0,24.0,NaN,NaN,NaN,NaN,NaN,INDIA: KASHMIR: SRINAGAR,34.1,74.8,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,NaN,NaN


In [0]:
row_count = df_cleaned.count()

print(f"Total number of rows in the dataset: {row_count}")


Total number of rows in the dataset: 5869


In [0]:

# Filter rows where critical earthquake data is NOT NULL
df_filtered = df_cleaned.dropna(subset=["Year", "Mag", "Latitude", "Longitude", "Location Name"])
print(f"Total number of valid earthquake records: {df_filtered.count()}")

df_filtered.toPandas().head(5)



Total number of valid earthquake records: 4488


,Year,Mo,Dy,Hr,Mn,Sec,Tsu,Vol,Location Name,Latitude,Longitude,Focal Depth (km),Mag,MMI Int,Deaths,Death Description,Missing,Missing Description,Injuries,Injuries Description,Damage ($Mil),Damage Description,Houses Destroyed,Houses Destroyed Description,Houses Damaged,Houses Damaged Description,Total Deaths,Total Death Description,Total Missing,Total Missing Description,Total Injuries,Total Injuries Description,Total Damage ($Mil),Total Damage Description,Total Houses Destroyed,Total Houses Destroyed Description,Total Houses Damaged,Total Houses Damaged Description
0,1501.0,1.0,19.0,NaN,NaN,NaN,NaN,NaN,CHINA: SHAANXI PROVINCE: CHAOYI,34.8,110.1,NaN,7.0,9.0,400.0,3.0,NaN,NaN,NaN,NaN,NaN,4.0,5400.0,4.0,NaN,NaN,400.0,3.0,NaN,NaN,NaN,NaN,NaN,4.0,5400.0,4.0,NaN,NaN
1,1502.0,10.0,17.0,NaN,NaN,NaN,NaN,NaN,CHINA: HENAN PROVINCE: PUCHENG,35.7,115.3,NaN,6.5,8.0,50.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,50.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1505.0,6.0,6.0,NaN,NaN,NaN,NaN,NaN,NEPAL-INDIA,29.5,83.0,NaN,8.2,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN
3,1505.0,10.0,16.0,NaN,NaN,NaN,NaN,NaN,"CHINA: SHANXI PROVINCE: ANYI, GUEHENG",35.2,110.9,NaN,5.0,6.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1507.0,11.0,4.0,NaN,NaN,NaN,NaN,NaN,"CHINA: YUNNAN PROVINCE: ANNING, YUXI",24.6,102.5,NaN,5.3,6.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
from pyspark.sql.functions import when, split, col, trim

# Extract country using ":" or "," as delimiters
df_with_country = df_filtered.withColumn(
    "Country",
    when(col("Location Name").contains(":"),
         split(col("Location Name"), ":")[0])  
    .when(col("Location Name").contains(","), 
         trim(split(col("Location Name"), ",")[1]))
    .when(col("Location Name").contains("-"), 
         trim(split(col("Location Name"), "-")[1]))
    .otherwise(col("Location Name"))  
)

# Show sample data with extracted Country
df_with_country.select("Location Name", "Country").show(10, truncate=False)


+------------------------------------------+----------+
|Location Name                             |Country   |
+------------------------------------------+----------+
|CHINA:  SHAANXI PROVINCE:  CHAOYI         |CHINA     |
|CHINA:  HENAN PROVINCE:  PUCHENG          |CHINA     |
|NEPAL-INDIA                               |INDIA     |
|CHINA:  SHANXI PROVINCE:  ANYI, GUEHENG   |CHINA     |
|CHINA:  YUNNAN PROVINCE:  ANNING, YUXI    |CHINA     |
|TURKEY:  ISTANBUL                         |TURKEY    |
|SEIONAIKAI, JAPAN                         |JAPAN     |
|JAPAN:  ENSHUNADA SEA                     |JAPAN     |
|BALKANS NW:  SLOVENIA:  IDRIJA,SKOFJA LOKA|BALKANS NW|
|CHINA:  YUNNAN PROVINCE                   |CHINA     |
+------------------------------------------+----------+
only showing top 10 rows



In [0]:
from pyspark.sql.functions import col

# Convert Year, Month, and Date to Integer (removes decimals) and rename columns
df_cleaned = df_with_country.withColumnRenamed("Mo", "Month") \
                            .withColumnRenamed("Dy", "Date") \
                            .withColumnRenamed("Total Deaths", "Total_Deaths") \
                            .withColumn("Year", col("Year").cast("int")) \
                            .withColumn("Month", col("Month").cast("int")) \
                            .withColumn("Date", col("Date").cast("int"))
df_cleaned = df_cleaned.drop("Location Name","Focal Depth (km)","MMI Int","Death Description","Missing Description","Injuries Description","Damage Description","Houses Destroyed Description","Houses Damaged Description","Total Death Description","Total Missing Description","Total Injuries Description","Total Damage Description","Total Houses Destroyed Description","Total Houses Damaged Description","Damage ($Mil)","Houses Destroyed","Houses Damaged")

# Show cleaned data
df_cleaned.toPandas().head(5)

,Year,Month,Date,Hr,Mn,Sec,Tsu,Vol,Latitude,Longitude,Mag,Deaths,Missing,Injuries,Total_Deaths,Total Missing,Total Injuries,Total Damage ($Mil),Total Houses Destroyed,Total Houses Damaged,Country
0,1501,1.0,19.0,NaN,NaN,NaN,NaN,NaN,34.8,110.1,7.0,400.0,NaN,NaN,400.0,NaN,NaN,NaN,5400.0,NaN,CHINA
1,1502,10.0,17.0,NaN,NaN,NaN,NaN,NaN,35.7,115.3,6.5,50.0,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,CHINA
2,1505,6.0,6.0,NaN,NaN,NaN,NaN,NaN,29.5,83.0,8.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INDIA
3,1505,10.0,16.0,NaN,NaN,NaN,NaN,NaN,35.2,110.9,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHINA
4,1507,11.0,4.0,NaN,NaN,NaN,NaN,NaN,24.6,102.5,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHINA


In [0]:
df_cleaned.write.option("mergeSchema", "true").option("spark.sql.extensions", "delta.sql.ColumnMappingSparkSessionExtensions").option("delta.columnMapping.mode", "name").mode("overwrite").saveAsTable("hive_metastore.default.earthquake_data_updated")
